In [ ]:
import os

# List the current directory
#print(os.listdir(.))


# Hi Ronen,
# 
#    How about the following scenario? Will the scheduled scrub be initiated or not? 
# 
#     osd_scrub_begin_week_day, osd_scrub_end_week_day are same.
#     osd_scrub_begin_hour = 4
#     osd_scrub_end_hour =2 
#     and the current time is = 3     
# 
# Regards,
# Bharath

In [ ]:
# Ask the user to select a digit in the range 0 to 8
k = int(input("Please select a digit in the range 0 to 8: "))

# Check if the digit is within the valid range
if 0 <= k <= 8:
    print(f"will be running in k{k}")
else:
    print("Invalid selection. Please select a digit between 0 and 8.")
    exit(1)

from pathlib import Path

wd = Path(f"/home/rfriedma/src/k{k}/ceph/build")
os.chdir(wd)
%env CEPH_JTEST_ROOT=/home/rfriedma/src/k{k}/ceph/build
!echo $CEPH_JTEST_ROOT > /tmp/jpath


In [ ]:
!ls -l
!bash -c MGR=0 ls -l


In [ ]:
%%bash
scrtch=to_"`date +%d_%H%M`"
echo $scrtch

MDS=0 MGR=1 OSD=3 MON=1 ../src/vstart.sh -n  --without-dashboard --msgr2 -X --memstore -o "memstore_device_bytes=68435456" -o "osd_op_queue=wpq"
sleep 2
bin/ceph -s

#bin/ceph tell osd.* config set debug_osd 20/20

bin/ceph config set global osd_pool_default_pg_autoscale_mode off
sleep 2

# disable rescheduling of the queue due to 'no-scrub' flags
#bin/ceph tell osd.* config set osd_scrub_backoff_ratio 0.9999

# initial set of global scrub scheduling parameters
bin/ceph tell osd.* config set osd_scrub_interval_randomize_ratio 0.1
bin/ceph tell osd.* config set osd_deep_scrub_randomize_ratio 0
bin/ceph tell osd.* config set osd_scrub_min_interval 10
bin/ceph tell osd.* config set osd_scrub_max_interval 2000
bin/ceph tell osd.* config set osd_deep_scrub_interval 600



# our specific time-of-day scrub scheduling parameters
# (setting the parameters so that no periodic scrubs are allowed now)
curh=`date '+%-H'`
allowed_from=$(( (curh + 2) % 24 ))
allowed_to=$(( (curh + 22) % 24 ))
bin/ceph tell osd.* config set osd_scrub_begin_hour $allowed_from
bin/ceph tell osd.* config set osd_scrub_end_hour $allowed_to


#PL1 is of size 3

bin/ceph osd pool create pl1 8 8
#bin/ceph osd pool autoscale-status
sleep 1
pl1_num=`bin/ceph osd pool stats pl1 | sed -n -r -e 's/.*id[^0-9]*([0-9]+)$/\1/p'`
echo $pl1_num
bin/ceph osd pool set pl1 size 3
bin/ceph osd pool set pl1 min_size 3
bin/ceph osd pool set pl1 pg_autoscale_mode off
bin/ceph osd pool stats
#bin/ceph osd pool set pl1 noscrub 1
#bin/ceph osd pool set pl1 nodeep-scrub 1
sleep 2

bin/rados bench -p pl1 -t 1 1 write -b 4096 --max-objects 8  --no-cleanup; 
bin/rados bench -p pl1 1 write -b 4096 --max-objects 128 --show-time --no-cleanup --run-name eeeee

# #PL2
# 
# bin/ceph osd pool create pl2 8 8
# bin/ceph osd pool set pl2 size 3
# bin/ceph osd pool set pl2 min_size 3
# bin/ceph osd pool set pl2 pg_autoscale_mode off
# bin/ceph osd pool stats
# bin/ceph osd pool set pl2 noscrub 1
# bin/ceph osd pool set pl2 nodeep-scrub 1
# sleep 2
# 
# bin/rados bench -p pl2 -t 1 1 write -b 4096 --max-objects 8  --no-cleanup; 
# bin/rados bench -p pl2 1 write -b 4096 --max-objects 128 --show-time --no-cleanup --run-name eeeee
sleep 1
bin/ceph tell osd.* config set debug_osd 20/20

In [ ]:
%%bash
echo '-------->' $CEPH_JTEST_ROOT
jp=$CEPH_JTEST_ROOT
if [ -z $CEPH_JTEST_ROOT ]; then
    echo "CEPH_JTEST_ROOT is not set"
    [[ -f /tmp/jpath ]] && jp=`cat /tmp/jpath` || jp='.'
    echo '-------->' $jp
    %env CEPH_JTEST_ROOT=$jp
fi
cd $jp


In [ ]:
%%bash

file_path="/tmp/jpath"
if [ -f "$file_path" ]; then
        file_contents=$(cat "$file_path")
        echo "File contents read into variable."
else
        echo "File does not exist."
fi

In [ ]:
raise SystemExit("Stop here")

# Termination


In [ ]:
%%bash
echo '-------->' $CEPH_JTEST_ROOT
cd $CEPH_JTEST_ROOT
if [ -z $CEPH_JTEST_ROOT ]; then
    echo "CEPH_JTEST_ROOT is not set"
    [[ -f /tmp/jpath ]] && jp=`cat /tmp/jpath` || jp='.'
    echo '-------->' $jp
    cd $jp
    %env CEPH_JTEST_ROOT=$jp
fi

pwd

../src/stop.sh
sleep 4
../src/stop.sh
